In [96]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [97]:
# initial conditions
x1_0 = 0
x2_0 = 800
x0 = [x1_0,x2_0]

In [98]:
#Successional Sequence I Differential Equations
def odes_seqI(x, t):
    x1, x2 = x
    
    if t == 0:
        dx1dt = 0
        dx2dt = 0
    else:
        if t < 25:
            a10 = 0.006 #fixed
            a02 = 0.0125 
            a12 = 0.009
            a21 = 0.1 #fixed
        else:
            a10 = 0.006 #fixed
            a02 = 0.004
            a12 = 0.0188
            a21 = 0.1 #fixed

        dx1dt = a10 * x1 - a21 * x1 + a12 * x2
        dx2dt = 10 - a10 * x1 + a21 * x1 - a12 * x2 - a02 * x2

    return [dx1dt, dx2dt]

In [99]:
#Successional Sequence II Differential Equations
def odes_seqII(x, t):
    x1, x2 = x
    
    if t == 0:
        dx1dt = 0
        dx2dt = 0
    else:
        if t < 300:
            a10 = 0.0003 #fixed
            a02 = 0.0125 
            a12 = 0.0003
            a21 = 0.00035 #fixed
        else:
            a10 = 0.0003 #fixed
            a02 = 0.004 
            a12 = 0.0002
            a21 = 0.00035 #fixed

        dx1dt = a10 * x1 - a21 * x1 + a12 * x2
        dx2dt = 10 - a10 * x1 + a21 * x1 - a12 * x2 - a02 * x2

    return [dx1dt, dx2dt]

In [104]:
#Succession Sequence I - Table 2
# Add a large time point for evaluation
times_to_evaluate = [0, 1, 24, 100, 200, 400, 600, 1000, 10000]
evolution_stages = ['BG', 'ES1', 'ES2', 'ES3', 'MS1', 'MS2', 'MS3', 'LS', 'SS']

# Evaluate the solution at specific time points
x_evaluated = odeint(odes_seqI, x0, times_to_evaluate)

#Other values
dxdt=[]
z10=[]
z20=[]
f21=[]
f12=[]
y02=[]
a10 = 0.006
a21 = 0.1
for i in range (len(times_to_evaluate)):
    dxdt.append(odes_seqI(x_evaluated [i], times_to_evaluate[i]))
    if i==0:
        z10.append(0)
        z20.append(10)
        f21.append(0)
        f12.append(0)
        y02.append(10)
    else:
        if i<3:
            a02 = 0.0125
            a12 = 0.009
        else:
            a02 = 0.004
            a12 = 0.0188

        z10.append(a10*x_evaluated [i,0])
        z20.append(10-a10*x_evaluated [i,0])
        f21.append(a21*x_evaluated [i,0])
        f12.append(a12*x_evaluated [i,1])
        y02.append(a02*x_evaluated[i, 1])

In [105]:
# Creating a DataFrame
data = {
    "Succession Stage": evolution_stages,
    "Time": times_to_evaluate,
    "z10": z10,
    "z20": z20,
    "f21": f21,
    "f12": f12,
    "x1": [x[0] for x in x_evaluated],
    "x2": [x[1] for x in x_evaluated],
    "x1_dot": [x[0] for x in dxdt],
    "x2_dot": [x[1] for x in dxdt],
    "y02": y02
}

df = pd.DataFrame(data)

# Display the DataFrame
print("Resulting DataFrame:")
print(df)

# Export to Excel
excel_filename = "result_data.xlsx"
df.to_excel(excel_filename, index=False)
print(f"\nData exported to {excel_filename}")

Resulting DataFrame:
  Succession Stage   Time       z10        z20        f21        f12  \
0               BG      0  0.000000  10.000000   0.000000   0.000000   
1              ES1      1  0.041050   9.958950   0.684174   7.138814   
2              ES2     24  0.387473   9.612527   6.457885   6.725270   
3              ES3    100  1.240357   8.759643  20.672619  20.405228   
4              MS1    200  1.736666   8.263334  28.944426  27.905365   
5              MS2    400  2.348760   7.651240  39.146003  37.156853   
6              MS3    600  2.664291   7.335709  44.404843  41.925929   
7               LS   1000  2.910791   7.089209  48.513184  45.651653   
8               SS  10000  3.000000   7.000000  50.000000  47.000000   

           x1           x2        x1_dot        x2_dot        y02  
0    0.000000   800.000000  0.000000e+00  0.000000e+00  10.000000  
1    6.841737   793.201575  6.495691e+00 -6.410711e+00   9.915020  
2   64.578846   747.252178  6.548581e-01  4.489689e-03

In [106]:
#Succession Sequence II - Table 2
# Add a large time point for evaluation
times_to_evaluate = [0, 40, 200, 500, 700, 4300, 20100, 49700, 1000000]
evolution_stages = ['BG', 'ES1', 'ES2', 'ES3', 'MS1', 'MS2', 'MS3', 'LS', 'SS']

# Evaluate the solution at specific time points
x_evaluated = odeint(odes_seqII, x0, times_to_evaluate)

#Other values
dxdt=[]
z10=[]
z20=[]
f21=[]
f12=[]
y02=[]
a10 = 0.0003
a21 = 0.00035
for i in range (len(times_to_evaluate)):
    dxdt.append(odes_seqII(x_evaluated [i], times_to_evaluate[i]))
    if i==0:
        z10.append(0)
        z20.append(10)
        f21.append(0)
        f12.append(0)
        y02.append(10)
    else:
        if i<3:
            a02 = 0.0125 
            a12 = 0.0003
        else:
            a02 = 0.004 
            a12 = 0.0002

        z10.append(a10*x_evaluated [i,0])
        z20.append(10-a10*x_evaluated [i,0])
        f21.append(a21*x_evaluated [i,0])
        f12.append(a12*x_evaluated [i,1])
        y02.append(a02*x_evaluated[i, 1])

In [107]:
# Creating a DataFrame
data = {
    "Succession Stage": evolution_stages,
    "Time": times_to_evaluate,
    "z10": z10,
    "z20": z20,
    "f21": f21,
    "f12": f12,
    "x1": [x[0] for x in x_evaluated],
    "x2": [x[1] for x in x_evaluated],
    "x1_dot": [x[0] for x in dxdt],
    "x2_dot": [x[1] for x in dxdt],
    "y02": y02
}

df = pd.DataFrame(data)

# Display the DataFrame
print("Resulting DataFrame:")
print(df)

# Export to Excel
excel_filename = "result_data2.xlsx"
df.to_excel(excel_filename, index=False)
print(f"\nData exported to {excel_filename}")

Resulting DataFrame:
  Succession Stage     Time       z10        z20       f21       f12  \
0               BG        0  0.000000  10.000000  0.000000  0.000000   
1              ES1       40  0.002862   9.997138  0.003340  0.237748   
2              ES2      200  0.014114   9.985886  0.016466  0.234845   
3              ES3      500  0.036381   9.963619  0.042445  0.338252   
4              MS1      700  0.058887   9.941113  0.068702  0.416861   
5              MS2     4300  0.518364   9.481636  0.604758  0.480079   
6              MS3    20100  1.830042   8.169958  2.135048  0.490608   
7               LS    49700  2.713996   7.286004  3.166328  0.497704   
8               SS  1000000  3.000000   7.000000  3.500000  0.500000   

            x1           x2        x1_dot        x2_dot        y02  
0     0.000000   800.000000  0.000000e+00  0.000000e+00  10.000000  
1     9.541565   792.494912  2.372714e-01 -1.434578e-01   9.906186  
2    47.046856   782.817356  2.324929e-01 -1.770982